In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
data = pd.read_csv('../input/housing-in-london/housing_in_london_monthly_variables.csv')

In [5]:
data

,date,area,average_price,code,houses_sold,no_of_crimes,borough_flag
0,1995-01-01,city of london,91449,E09000001,17.0,NaN,1
1,1995-02-01,city of london,82203,E09000001,7.0,NaN,1
2,1995-03-01,city of london,79121,E09000001,14.0,NaN,1
3,1995-04-01,city of london,77101,E09000001,7.0,NaN,1
4,1995-05-01,city of london,84409,E09000001,10.0,NaN,1
...,...,...,...,...,...,...,...
13544,2019-09-01,england,249942,E92000001,64605.0,NaN,0
13545,2019-10-01,england,249376,E92000001,68677.0,NaN,0
13546,2019-11-01,england,248515,E92000001,67814.0,NaN,0
13547,2019-12-01,england,250410,E92000001,NaN,NaN,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           13549 non-null  object 
 1   area           13549 non-null  object 
 2   average_price  13549 non-null  int64  
 3   code           13549 non-null  object 
 4   houses_sold    13455 non-null  float64
 5   no_of_crimes   7439 non-null   float64
 6   borough_flag   13549 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 741.1+ KB


In [7]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop redundant columns
    df = df.drop('code', axis=1)
    
    # Drop columns with too many missing values
    df = df.drop('no_of_crimes', axis=1)
    
    # Drop rows with missing target values
    missing_target_rows = df[df['houses_sold'].isna()].index
    df = df.drop(missing_target_rows, axis=0).reset_index(drop=True)
    
    # Extract date features
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df = df.drop('date', axis=1)
    
    # One-hot encode the area column
    area_dummies = pd.get_dummies(df['area'], prefix='area')
    df = pd.concat([df, area_dummies], axis=1)
    df = df.drop('area', axis=1)
    
    # Split df into X and y
    y = df['houses_sold']
    X = df.drop('houses_sold', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    
    # Scale X
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

In [8]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

In [11]:
models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor(),
    "                               XGBoost": XGBRegressor(),
    "                              LightGBM": LGBMRegressor(),
    "                              CatBoost": CatBoostRegressor(verbose=0)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.
                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.
                               XGBoost trained.
                              LightGBM trained.
                              CatBoost trained.


In [12]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    print(name + " RMSE: {:.4f}".format(rmse))
    
    
for name, model in models.items():
    y_pred = model.predict(X_test)
    r2 = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
    print(name + " R^2: {:.5f}".format(r2))


                     Linear Regression RMSE: 3336.2444
 Linear Regression (L2 Regularization) RMSE: 3336.2725
 Linear Regression (L1 Regularization) RMSE: 3336.4065
                   K-Nearest Neighbors RMSE: 2264.7955
                        Neural Network RMSE: 3348.6627
Support Vector Machine (Linear Kernel) RMSE: 10322.1869
   Support Vector Machine (RBF Kernel) RMSE: 12501.5496
                         Decision Tree RMSE: 1932.8505
                         Random Forest RMSE: 1487.1916
                     Gradient Boosting RMSE: 1741.2665
                               XGBoost RMSE: 1287.5804
                              LightGBM RMSE: 1410.0693
                              CatBoost RMSE: 1295.0794
                     Linear Regression R^2: 0.92373
 Linear Regression (L2 Regularization) R^2: 0.92373
 Linear Regression (L1 Regularization) R^2: 0.92372
                   K-Nearest Neighbors R^2: 0.96485
                        Neural Network R^2: 0.92316
Support Vector Machine 

In [13]:
############### Df #########################
modell = []
rmsel = []
r2l = []
for name, model in models.items():
    y_pred = model.predict(X_test)
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))
#     print(name + " RMSE: {:.4f}".format(rmse))
    r2 = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
#     print(name + " R^2: {:.5f}".format(r2))
    modell.append(name)
    rmsel.append(rmse)
    r2l.append(r2)
df = pd.DataFrame({'ModelName': modell, 'RMSE': rmsel, 'R_squared': r2l})
df


,ModelName,RMSE,R_squared
0,Linear Regression,3336.244437,0.923728
1,Linear Regression (L2 Regularization),3336.272512,0.923727
2,Linear Regression (L1 Regularization),3336.406543,0.923721
3,K-Nearest Neighbors,2264.795482,0.964852
4,Neural Network,3348.662697,0.923159
5,Support Vector Machine (Linear Kernel),10322.186907,0.269885
6,Support Vector Machine (RBF Kernel),12501.549564,-0.070965
7,Decision Tree,1932.850507,0.974400
8,Random Forest,1487.191636,0.984844
9,Gradient Boosting,1741.266477,0.979223
